# LeNet-5

In [ ]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from keras.layers import Dense, Dropout
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Flatten

In [ ]:
(X_train, y_train), (X_valid, y_valid) = mnist.load_data()

11501568/11490434 [==============================] - 0s 0us/step


In [ ]:
print(X_train.shape)
print(X_valid.shape)

(60000, 28, 28)
(10000, 28, 28)


### Converting the digits from integers to floats so that they are scaled to range from 0 to 1

- Since MNIST images are black and white, there's only one dimension, so a reshape is needed

In [ ]:
X_train = X_train.reshape(60000, 28, 28, 1).astype('float32')
X_valid = X_valid.reshape(10000, 28, 28, 1).astype('float32')

In [ ]:
X_train /= 255
X_valid /= 255

### Converting the y labels to one-hot encodings

In [ ]:
n_classes = 10
y_train = to_categorical(y_train, n_classes)
y_valid = to_categorical(y_valid, n_classes)

## Creating the Model

In [ ]:
model = Sequential()

# First Convolutional Layer (32 filters, kernels)
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu',
                    input_shape=(28, 28, 1)))

# Second Convolutional Layer (64 filters, kernels), with Pooling and Dropout
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
# Flatten converts the three-dimensional activation map output by Conv2D() 
# to a one-dimensional array. This enables us to feed the activations as 
# inputs into a Dense layer, which can only accept one-dimensional arrays.
model.add(Flatten())

# Dense Hidden Layer, with Dropout
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

# Output Layer
model.add(Dense(n_classes, activation='softmax'))

- Activation Map = $\frac{D - F + 2P}{S} + 1$
<br>
<br>
* Knowing that:
    * Stride S = 1
    * No Padding, P = 0
    * Image Size D = 28
    * Kernel size F = 3
<br>
<br>

Activation Map = $\frac{28 - 3 + 2x0}{1} + 1$
<br>
<br>
Activation Map = 26
<br>
<br>
- padding='same' guarantees that the activation map will have the same dimensions as the input



- The two convolutional layers learn to identify and label spatial features in the images, and these spatial features are then fed into a dense layer that maps these spatial features to a particular class of images. 
- **The convolutional layers can be thought of as feature extractors**

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 64)        18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 12, 12, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 12, 12, 64)        0         
                                                                 
 flatten (Flatten)           (None, 9216)              0         
                                                                 
 dense (Dense)               (None, 128)               1179776   
                                                        

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='nadam', metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, batch_size=128, epochs=10, verbose=1, validation_data=(X_valid, y_valid))

Epoch 1/10
469/469 [==============================] - 18s 11ms/step - loss: 0.2307 - accuracy: 0.9297 - val_loss: 0.0495 - val_accuracy: 0.9839
Epoch 2/10
469/469 [==============================] - 5s 11ms/step - loss: 0.0845 - accuracy: 0.9754 - val_loss: 0.0425 - val_accuracy: 0.9860
Epoch 3/10
469/469 [==============================] - 5s 10ms/step - loss: 0.0626 - accuracy: 0.9813 - val_loss: 0.0322 - val_accuracy: 0.9887
Epoch 4/10
469/469 [==============================] - 5s 11ms/step - loss: 0.0508 - accuracy: 0.9842 - val_loss: 0.0316 - val_accuracy: 0.9898
Epoch 5/10
469/469 [==============================] - 5s 11ms/step - loss: 0.0445 - accuracy: 0.9863 - val_loss: 0.0298 - val_accuracy: 0.9899
Epoch 6/10
469/469 [==============================] - 5s 11ms/step - loss: 0.0376 - accuracy: 0.9883 - val_loss: 0.0374 - val_accuracy: 0.9877
Epoch 7/10
469/469 [==============================] - 5s 11ms/step - loss: 0.0330 - accuracy: 0.9900 - val_loss: 0.0275 - val_accuracy: 0.990

# AlexNet

In [ ]:
from keras.layers import BatchNormalization

In [ ]:
model = Sequential()

# First Conv-Pool Block
model.add(Conv2D(96, kernel_size=(11, 11), strides=(4, 4), activation='relu',
          input_shape=(224, 224, 3)))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(BatchNormalization())

# Second Conv-Pool Block
model.add(Conv2D(256, kernel_size=(5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(BatchNormalization())

# Third Conv-Pool Block
model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(384, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(384, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(BatchNormalization())

# Dense Layers
model.add(Flatten())
model.add(Dense(4096, activation='tanh'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='tanh'))
model.add(Dropout(0.5))

# Output Layer
model.add(Dense(17, activation='softmax'))

# Object Detection

In [ ]:
from keras.applications.vgg19 import VGG19
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Load the pre-trained VGG19 model
vgg19 = VGG19(include_top=False,
              weights="imagenet", # load imagenet data parameters trained
              input_shape=(224, 224, 3),
              pooling=None)

# Freeze all the layers in the base VGGNet19 model
# The parameters in these layers will not be updated during training.
for layer in vgg19.layers:
    layer.trainable = False

In [ ]:
# Instantiate the sequential model and add the VGG19 model
model = Sequential()
model.add(vgg19)

# Add the custom layers atop the VGG19 model
model.add(Flatten(name="flattened"))
model.add(Dropout(0.5, name="dropout"))
model.add(Dense(2, activation="softmax", name="predictions"))

# Compile the model for training
model.compile(optimizer='adam', loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [ ]:
! wget -c https://www.dropbox.com/s/86r9z1kb42422up/hot-dog-not-hot-dog.tar.gz
! tar -xzf hot-dog-not-hot-dog.tar.gz

--2022-09-26 23:10:27--  https://www.dropbox.com/s/86r9z1kb42422up/hot-dog-not-hot-dog.tar.gz
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/86r9z1kb42422up/hot-dog-not-hot-dog.tar.gz [following]
--2022-09-26 23:10:27--  https://www.dropbox.com/s/raw/86r9z1kb42422up/hot-dog-not-hot-dog.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 404 Not Found
2022-09-26 23:10:27 ERROR 404: Not Found.

tar (child): hot-dog-not-hot-dog.tar.gz: Cannot open: No such file or directory
tar (child): Error is not recoverable: exiting now
tar: Child returned status 2
tar: Error is not recoverable: exiting now


In [ ]:
# Instantiate two image generator classes
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    data_format='channels_last',
    rotation_range=30,
    horizontal_flip=True,
    fill_mode='reflect')

valid_datagen = ImageDataGenerator(
    rescale=1.0/225,
    data_format='channels_last')

# Define the batch size
batch_size = 32

# Define the train and validation generators: 
train_generator = train_datagen.flow_from_directory(
    directory='./hot-dog-not-hot-dog/train',
    target_size=(224, 224),
    classes=['hot_dog','not_hot_dog'],
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    seed=42)

valid_generator = valid_datagen.flow_from_directory(
    directory='./hot-dog-not-hot-dog/test',
    target_size=(224, 224),
    classes=['hot_dog','not_hot_dog'],
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    seed=42)

Found 0 images belonging to 2 classes.
Found 0 images belonging to 2 classes.


In [ ]:
model.fit_generator(train_generator, steps_per_epoch=15, 
                    epochs=16, validation_data=valid_generator, 
                    validation_steps=15)